# Moving Average Crossover Strategy

We are going to develop a moving average cross over strategy. The idea here is that when the moving averages cross over each other, this is a sign that there might be momentum in the market. 

The general rules around the strategy: 
* If the short term moving average is above the long term moving average, we go long
* If the short term moving average is below the long term moving average and we have a position already, we close all of it and wait until the next signal

The general flow of the main script will be: 
* Get the market data
* Compute your signal - moving averages
* Check positions 
* Do position handling based on your signals and current positions

What types of logging do we need? 
* Log of the

In [5]:
import logging 
import os
import pandas as pd 
import numpy as np 

from datetime import datetime, timedelta
# All the Alpaca Loaders
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from dotenv import load_dotenv

dotenv_path = r'C:\Users\srerr\Documents\Projects\PersonalProjects\stonks\alpaca\.env'

In [7]:
load_dotenv()
api_key = os.environ['ALPACA_PAPER_TRADING_KEY_ID']
secret_key = os.environ['ALPACA_PAPER_TRADING_SECRET_KEY']

In [1]:
class MovingAverageCrossOverStrategy:


    def __init__(self, symbol: str, short_period: int, long_period: int, alpaca_api_key: str, alpaca_secret_key: str, capital: float) -> None:

        self.capital = capital
        self.symbol = symbol
        self.short_period = short_period
        self.long_period = long_period
        self.alpaca_api_key = alpaca_api_key
        self.alpaca_secret_key = alpaca_secret_key
        self.positions = []
        self.trades = []
        self.total_pnl = 0

        self.trading_client = TradingClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key)
        self.stock_data_client = StockHistoricalDataClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key)
        self.crypto_data_client = CryptoHistoricalDataClient
        self.start_date = datetime.now()
        self.end_date = None
        self.historical_data = self.get_historical_data()


    def get_historical_data(self):
        data_start_time = self.start_date - timedelta(days = self.long_period + 100)
        market_data_request = StockBarsRequest(
            symbol_or_symbols=self.symbol,
            start = data_start_time,
            timeframe=TimeFrame.Day
        )

        df = self.stock_data_client.get_stock_bars(market_data_request).df

        return df

    def generate_signal(self):
        """Takes the historical data and generates a buy, sell, or hold signal

        Returns:
            str: Signal which tells us what to do based on the data and current position
        """

        historical_bars = self.get_historical_data()

        # Generating the moving averages
        long_sma_column_mame = f'{self.long_period}-day SMA'
        short_sma_column_name = f'{self.short_period}-day SMA'

        historical_bars[long_sma_column_mame] = historical_bars['close'].rolling(self.long_period).mean()
        historical_bars[short_sma_column_name] = historical_bars['close'].rolling(self.short_period).mean()

        # We want to check for a crossover, only then should we go short or long
        # 1. Shorter moving average crosses above the longer one
        # 2. Shorter moving average crosses below the longer one

        prev_short_moving_average = historical_bars[short_sma_column_name].iloc[-2]
        current_short_moving_average = historical_bars[short_sma_column_name].iloc[-1]

        prev_long_moving_average = historical_bars[long_sma_column_mame].iloc[-2]
        current_long_moving_average = historical_bars[long_sma_column_mame].iloc[-1]

        if (prev_short_moving_average > prev_long_moving_average) and (current_short_moving_average < current_long_moving_average):
            ## We have a cross going down and we should go short

            signal = OrderSide.SELL

        elif (prev_short_moving_average < prev_long_moving_average) and (current_short_moving_average > current_long_moving_average):

            # We have a cross going up and we should go long
            signal = OrderSide.BUY

        else:

            signal = None

        return signal

    def calculate_shares(self):
        assert not self.historical_data.empty(), "Historical data has not been run yet?"
        shares = self.capital // self.historical_data['vwap'].iloc[-1]

        return shares

    def run(self):
        """Goes through the process of running the strategy and everything involved
        """

        signal = self.generate_signal()
        shares_to_trade = self.calculate_shares()

        if signal == OrderSide.BUY or signal == OrderSide.SELL:
            if len(self.positions) == 0:

                market_order_request = MarketOrderRequest(
                    symbol = self.symbol,
                    qty = shares_to_trade,
                    side = signal,
                    time_in_force = TimeInForce.DAY
                )

                order = self.trading_client.submit_order(market_order_request)

                #TODO Take this order object and create a new AlpacaTrade object with it's information

            else:
                current_position = self.positions[0]
                amount_to_close = current_position.quantity

                if (current_position.side == 'Long' and signal == OrderSide.SELL) or (current_position.side == "Short" and signal == OrderSide.BUY):
                    # In this case, we have to close our long position and go short

                    market_order_close_out = self.create_and_submit_market_order(
                        quantity=amount_to_close,
                        side = signal
                    )

                    market_order_entry = self.create_and_submit_market_order(quantity=amount_to_close, side = signal)

                    #TODO Make sure to interact with the new alpaca position information


    def create_and_submit_market_order(self, quantity: int, side: OrderSide) -> Order:
        """Creates and submits a market order and returns the order object for you

        Args:
            quantity (int): Quantity to trade, either shares or tokens
            side (OrderSide): Orderside which you want to sell

        Returns:
            Order: Order object
        """

        request_params = MarketOrderRequest(
            symbol = self.symbol,
            qty = quantity,
            side = side,
            time_in_force = TimeInForce.DAY
        )

        order = self.trading_client.submit_order(order_data = request_params)

        return order

NameError: name 'OrderSide' is not defined

In [58]:
strategy = MovingAverageCrossOverStrategy(
    fast_period=10, 
    slow_period=40, 
    trading_pair='ETH/USD', 
    column = 'close', 
    api_key=api_key, 
    secret_key=secret_key
)

In [59]:
df = strategy.get_historical_data()

In [60]:
strategy.compute_signals(df = df)

'long'

In [ ]:
def main(): 

    trading_pair = 'ETH/USD'
    capital = 5000
    fast_period = 10 
    slow_period = 60
    column = 'close'
    api_key = api_key
    secret_key = secret_key

    # Instantiating the object
    strategy = MovingAverageCrossOverStrategy(
        fast_period=fast_period, 
        slow_period=slow_period, 
        trading_pair=trading_pair, 
        column=column, 
        api_key=api_key,
        secret_key=secret_key
    )

    # Get the historical 